In [6]:
import pandas as pd
import datetime
import csv
import os
import requests
import datetime as dt
import time

In [9]:
URL = "https://api.pushshift.io/reddit/submission/search" #query submission
PARAMS = {
    'after': 1596240000, #after August 1, 2020
    'before': 1597881600, #before August 20, 2020
    'sort_type': 'score',
    'sort': 'asc',
    'subreddit': 'Agriculture',
    'size': 20, #give only 20 search results
#      'fields': ["id","title","score","num_comments","created_utc"]    
}

#use the requests library to query pushshift api
r = requests.get(url = URL, params= PARAMS)
#parse returned data to a json object
r.json()

{'data': [{'all_awardings': [],
   'allow_live_comments': False,
   'author': 'adinaespinoza',
   'author_flair_css_class': None,
   'author_flair_richtext': [],
   'author_flair_text': None,
   'author_flair_type': 'text',
   'author_fullname': 't2_h9lpl7k',
   'author_patreon_flair': False,
   'author_premium': False,
   'awarders': [],
   'can_mod_post': False,
   'contest_mode': False,
   'created_utc': 1596321074,
   'domain': 'yahoo.com',
   'full_link': 'https://www.reddit.com/r/Agriculture/comments/i217jx/a_louisiana_woman_who_planted_seeds_from_a/',
   'gildings': {},
   'id': 'i217jx',
   'is_crosspostable': False,
   'is_meta': False,
   'is_original_content': False,
   'is_reddit_media_domain': False,
   'is_robot_indexable': False,
   'is_self': False,
   'is_video': False,
   'link_flair_background_color': '',
   'link_flair_richtext': [],
   'link_flair_text_color': 'dark',
   'link_flair_type': 'text',
   'locked': False,
   'media_only': False,
   'no_follow': True,
  

In [18]:
def to_utc(date):
    #this function converts an object to UTC--to automate the conversion
    #of dates instead of going to https://www.unixtimeconverter.io/
    return int(date.replace(tzinfo=dt.timezone.utc).timestamp())

def to_readable_date(timestamp):
    #this function converts the UTC format to a Year-Month-Day format
    return dt.datetime.fromtimestamp(timestamp).strftime("%Y-%m-%d")

#Declare start and end of reddit posts to extract
start_date = dt.datetime.strptime("2020-08-01", "%Y-%m-%d")
end_date = dt.datetime.strptime("2020-08-20", "%Y-%m-%d")

#Create a range of dates to iterate
#Note: Periods here represents the number of days it will create from the start date
#We also do a +2 since it will only generate up to August 19. We may include include August 21
#Since i want to get data from August 20 to August 21
date_range = (pd.date_range(
                start_date,
                periods=(end_date - start_date).days + 2)
             .tolist())

#prepare the parameters needed to call the API
sort_type="score"
sort="asc"
fields=["id","title","score","num_comments","created_utc"]
subreddit = 'Agriculture'
url = "https://api.pushshift.io/reddit/submission/search"
results = []
#loop through the dates
for i, s_date in enumerate(date_range):
    #prevent from getting an index out of range error
    if i != len(date_range)-1:
        #declare end date
        e_date = date_range[i+1]
        #call the API
        r = requests.get(url = url, params={
            'after': to_utc(s_date),
            'before': to_utc(e_date),
            'sort_type': sort_type,
            'sort': sort,
            'subreddit': subreddit,
            'fields': fields,
            "size": 500
        })
        
        #add logs
        print(f"Doing {s_date.strftime('%Y-%m-%d')} to {e_date.strftime('%Y-%m-%d')}")
        if r.status_code == 200:
            results.append(r.json()['data'])
            print("====Done")
        else:
            print("====Skipped")
        #so that we dont get blocked from abusing the API we call it after pausing for 1 second
        time.sleep(1)

Doing 2020-08-01 to 2020-08-02
====Done
Doing 2020-08-02 to 2020-08-03
====Done
Doing 2020-08-03 to 2020-08-04
====Done
Doing 2020-08-04 to 2020-08-05
====Done
Doing 2020-08-05 to 2020-08-06
====Done
Doing 2020-08-06 to 2020-08-07
====Done
Doing 2020-08-07 to 2020-08-08
====Done
Doing 2020-08-08 to 2020-08-09
====Done
Doing 2020-08-09 to 2020-08-10
====Done
Doing 2020-08-10 to 2020-08-11
====Done
Doing 2020-08-11 to 2020-08-12
====Done
Doing 2020-08-12 to 2020-08-13
====Done
Doing 2020-08-13 to 2020-08-14
====Done
Doing 2020-08-14 to 2020-08-15
====Done
Doing 2020-08-15 to 2020-08-16
====Done
Doing 2020-08-16 to 2020-08-17
====Done
Doing 2020-08-17 to 2020-08-18
====Done
Doing 2020-08-18 to 2020-08-19
====Done
Doing 2020-08-19 to 2020-08-20
====Done
Doing 2020-08-20 to 2020-08-21
====Done


In [19]:
results

[[{'created_utc': 1596321074,
   'id': 'i217jx',
   'num_comments': 0,
   'score': 1,
   'title': "A Louisiana woman who planted seeds from a mysterious package says she's not worried. But the USDA is, and it's asking people not to plant these unknown seeds."},
  {'created_utc': 1596288377,
   'id': 'i1s6du',
   'num_comments': 0,
   'score': 1,
   'title': 'Case Study on the Agricultural Sector in Jamaica'},
  {'created_utc': 1596281460,
   'id': 'i1qr9s',
   'num_comments': 0,
   'score': 1,
   'title': 'Check this out! 2x KRONE BIG M'},
  {'created_utc': 1596275096,
   'id': 'i1pnlm',
   'num_comments': 0,
   'score': 1,
   'title': 'This is simply amazing!'}],
 [{'created_utc': 1596374924,
   'id': 'i2c1wo',
   'num_comments': 0,
   'score': 1,
   'title': 'The red carpet'},
  {'created_utc': 1596342075,
   'id': 'i264np',
   'num_comments': 15,
   'score': 1,
   'title': 'Indoor / Vertical Farming under Artificial Light'},
  {'created_utc': 1596394153,
   'id': 'i2hccp',
   'num_c

In [26]:
flat_list = []
#loop through the reddit results
for sublist in results:
    #check if sublist is not empty. The reason we have empty list is because there are days wherein there are no submissions
    if sublist is not None:
        #for each dictionary in the sublist add it to the flat list
        for item in sublist:
            flat_list.append(item)

#pandas has a useful function called from_dict which will convert a list of dictionary objects into a dataframe
df = pd.DataFrame.from_dict(flat_list)
display(df.head())
df.to_csv("reddit_Agriculture.csv")        

,created_utc,id,num_comments,score,title
0,1596321074,i217jx,0,1,A Louisiana woman who planted seeds from a mys...
1,1596288377,i1s6du,0,1,Case Study on the Agricultural Sector in Jamaica
2,1596281460,i1qr9s,0,1,Check this out! 2x KRONE BIG M
3,1596275096,i1pnlm,0,1,This is simply amazing!
4,1596374924,i2c1wo,0,1,The red carpet


In [1]:
import pandas as pd

In [20]:
import pandas as pd
df = pd.read_csv("reddit_Agriculture.csv")
df

,Unnamed: 0,created_utc,id,num_comments,score,title
0,0,1596321074,i217jx,0,1,A Louisiana woman who planted seeds from a mys...
1,1,1596288377,i1s6du,0,1,Case Study on the Agricultural Sector in Jamaica
2,2,1596281460,i1qr9s,0,1,Check this out! 2x KRONE BIG M
3,3,1596275096,i1pnlm,0,1,This is simply amazing!
4,4,1596374924,i2c1wo,0,1,The red carpet
...,...,...,...,...,...,...
136,136,1597867433,icv80x,5,1,Can we make artificial plant roots/nutrient fe...
137,137,1597891626,id27to,0,1,Top 10 Countries with Rice Production
138,138,1597902478,id4oh8,0,1,Smart Harvest Market by Site of Operation (On-...
139,139,1597900868,id4cnc,0,1,Cotton Processing Market worth $72.6 billion b...


In [21]:
df.shape

(141, 6)

In [22]:
df.dtypes

Unnamed: 0       int64
created_utc      int64
id              object
num_comments     int64
score            int64
title           object
dtype: object

In [24]:
df['id'].value_counts()

id4oh8    1
i6gadz    1
i5hgfq    1
i7oqp1    1
i70b9d    1
         ..
i217jx    1
i4j0b3    1
i3dxdd    1
ibp10l    1
i49kl2    1
Name: id, Length: 141, dtype: int64

In [25]:
df['num_comments'].value_counts()

0     117
8       3
4       3
2       2
5       2
9       2
21      2
16      1
15      1
27      1
18      1
6       1
22      1
3       1
25      1
1       1
28      1
Name: num_comments, dtype: int64

In [26]:
df['score'].value_counts()

1    141
Name: score, dtype: int64

In [27]:
df['created_utc'].value_counts()

1596899836    1
1597900868    1
1597396556    1
1597513291    1
1597502282    1
             ..
1597201038    1
1596637360    1
1597202354    1
1596672179    1
1597138433    1
Name: created_utc, Length: 141, dtype: int64

In [28]:
df['title'].value_counts()

Quote of the Day                                                                                                                                                   2
Agricultural Machinery, mini combine harvester, forage harvester, big combine harvester, Farm Implements Manufacturers India - Gomselmash India Private Limited    2
10 Vital Roles of Algae in relation to Agriculture                                                                                                                 1
Be Prepared For The Expensive Tea!                                                                                                                                 1
Agricultural Packaging Market by Product, Application, Region - 2023                                                                                               1
                                                                                                                                                                  ..
Check this